# Benchmark EDUC at Large Scale

This notebook aims to benchmark the economic dispatch and unit commitment problems at large scale.

In [1]:
import pandas as pd

import ams

import ams.benchmarks as bp

from andes.utils.misc import elapsed

# Configure AMS logger
ams.config_logger(stream_level=20)

# Display tool versions
_ = bp.get_tool_versions()

ams.config_logger(stream_level=50)

Last run time: 2024-11-05 18:55:10
Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:27:15) [Clang 11.1.0 ]

Tool        Version                
----------  -----------------------
ltbams      0.9.10.post18+g43b7dbe3
cvxpy       1.5.3                  
pandapower  2.14.11                
PYPOWER     5.1.17                 
gurobipy    11.0.3                 
mosek       10.2.6                 
ecos        2.0.14                 
scs         3.2.7                  
piqp        0.4.2                  
numba       0.60.0                 


In [2]:
%matplotlib inline

In [3]:
def test_educ(case, routine='UC', solver='GUROBI', ignore_dpp=True, ):
    """
    Test the execution time of the specified routine on the given case.

    Parameters
    ----------
    case : str
        The path to the case file.
    routine : str, optional
        The name of the routine to test. Defaults to 'DCOPF'.
    solver : str, optional
        The name of the solver to use. Defaults to 'GUROBI'
    ignore_dpp : bool, optional
        Whether to ignore DPP. Defaults to True.

    Returns
    -------
    tuple
        A tuple containing the list of times and the list of objective values.
    """
    sp = ams.load(case, setup=True, default_config=True, no_output=True)

    rtn = sp.routines[routine]

    # Initialize AMS
    # --- matrices build ---
    t_mats, _ = elapsed()
    sp.mats.build()
    _, s_mats = elapsed(t_mats)

    # --- code generation ---
    t_parse, _ = elapsed()
    rtn.om.parse()
    _, s_parse = elapsed(t_parse)

    # --- code evaluation ---
    t_evaluate, _ = elapsed()
    rtn.om.evaluate()
    _, s_evaluate = elapsed(t_evaluate)

    # --- problem finalization ---
    t_finalize, _ = elapsed()
    rtn.om.finalize()
    _, s_finalize = elapsed(t_finalize)

    # --- rest init process ---
    t_postinit, _ = elapsed()
    rtn.init()
    _, s_postinit = elapsed(t_postinit)

    # --- run solvers ---
    t_ams_solve, _ = elapsed()
    rtn.run(solver=solver, ignore_dpp=ignore_dpp)
    _, s_ams_solve = elapsed(t_ams_solve)

    time = [s_mats, s_parse, s_evaluate, s_finalize,
            s_postinit, s_ams_solve]
    time = [float(t.split(' ')[0]) for t in time]

    return time

In [4]:
# run a small case to ensure test is working
_case0 = ams.get_case('ieee14/ieee14_uced.xlsx')
test_educ(_case0, routine='ED', solver='GUROBI', ignore_dpp=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25


[0.0007, 0.0063, 0.0064, 0.0001, 0.0001, 0.0451]

In [5]:
case = '../UCCase/goc31777_uced.xlsx'
t_ed = test_educ(case, routine='ED', solver='GUROBI', ignore_dpp=True)

In [6]:
case = '../UCCase/goc31777_uced.xlsx'
t_uc = test_educ(case, routine='UC', solver='GUROBI', ignore_dpp=True)

In [7]:
# Define the column names and the results
cols = ['ams_mats', 'ams_parse', 'ams_eval',
        'ams_final', 'ams_postinit', 'ams_solver']

# Create a DataFrame
df = pd.DataFrame({'time': cols, 'UC': t_uc, 'ED': t_ed})

df

,time,UC,ED
0,ams_mats,0.2223,0.2238
1,ams_parse,14.8685,19.6837
2,ams_eval,39.3289,48.6910
3,ams_final,0.0001,0.0001
4,ams_postinit,0.0007,0.0001
5,ams_solver,45.2441,39.0214


In [8]:
df.to_csv('../results_educ_large.csv')